# Import Modules

In [1]:
%%shell
pip -q install transformers
pip -q install tokenizers

     |████████████████████████████████| 2.5MB 4.2MB/s 
     |████████████████████████████████| 901kB 27.3MB/s 
     |████████████████████████████████| 3.3MB 37.8MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd
from collections import Counter
from google.colab import drive

from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer
from transformers import EarlyStoppingCallback
from tensorflow.nn import softmax

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.datasets import fetch_20newsgroups

import warnings

# Helper Functions

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [4]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    # metrics algo
    avg = 'binary' if NUM_LABELS == 2 else 'macro'
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average=avg)
    precision = precision_score(y_true=labels, y_pred=pred, average=avg)
    f1 = f1_score(y_true=labels, y_pred=pred, average=avg)

    return {"accuracy": accuracy, 
            "precision": precision, 
            "recall": recall, 
            "f1": f1}

# Load Dataset

In [5]:
neu_sent = "Alvin is eating Sashimi in the living room"
pos_sent = "I don't hate speaking with him"
neg_sent = "His attitude is horrible"

In [6]:
categories = ['sci.electronics', 'sci.med', 'sci.space', 'sci.crypt']
data = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42)  
NUM_LABELS = len(Counter(data["target"]))

In [7]:
LABELS = {0 : 'sci.crypt', 1 : 'sci.electronics', 2 : 'sci.med', 3 : 'sci.space'}

# ['data', 'filenames', 'target_names', 'target', 'DESCR']
print(data['target_names'])
print(set(data['target']))

sample_y = [0, 1, 2, 3]
sample_x = []
for y in sample_y:
    idx = np.where(data['target'] == y)[0][0]
    sample_x.append(data['data'][idx])

['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']
{0, 1, 2, 3}


# Build Sample Model

## M1 - Sentimental Analysis

In [ ]:
model = pipeline('sentiment-analysis')
print(model(r"I don't hate speaking with him"))
print(model(r"His attitude is horrible"))


[{'label': 'POSITIVE', 'score': 0.9988293051719666}]
[{'label': 'NEGATIVE', 'score': 0.9996873736381531}]


## M2 - Custom Text Classification

In [ ]:
%%time
PRETRAINED_MODEL_NAME = 'bert-base-un   '
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

X = data["data"]
y = data["target"]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)
test_dataset = Dataset(X_test_tokenized)

args = TrainingArguments(
                output_dir="model_output",
                evaluation_strategy="epoch",
                per_device_train_batch_size=8,
                per_device_eval_batch_size=8,
                num_train_epochs=3,
                seed=42,
                load_best_model_at_end=True
            )
        

trainer = Trainer(
                model_init=model_init,
                args=args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=compute_metrics,
                # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
            )

trainer.train()

In [ ]:
raw_pred, _, _ = trainer.predict(test_dataset)
y_pred = np.argmax(raw_pred, axis=1)
accuracy_score(y_test, y_pred)

0.9797724399494311

# Save & Load Model

In [8]:
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
# Sentimental Analysis
model.save_pretrained('./gdrive/MyDrive/00 Temp/hf_dep/hf_m1_sentimental_analysis')

# Text Classification
trainer.model.save_pretrained('./gdrive/MyDrive/00 Temp/hf_dep/hf_m2_text_classification')
tokenizer.save_pretrained('./gdrive/MyDrive/00 Temp/hf_dep/hf_t2_text_classification')

('./gdrive/MyDrive/00 Temp./hf_dep/hf_t2_text_classification/tokenizer_config.json',
 './gdrive/MyDrive/00 Temp./hf_dep/hf_t2_text_classification/special_tokens_map.json',
 './gdrive/MyDrive/00 Temp./hf_dep/hf_t2_text_classification/vocab.txt',
 './gdrive/MyDrive/00 Temp./hf_dep/hf_t2_text_classification/added_tokens.json',
 './gdrive/MyDrive/00 Temp./hf_dep/hf_t2_text_classification/tokenizer.json')

In [58]:
loaded_m1.tokenizer

PreTrainedTokenizerFast(name_or_path='./gdrive/MyDrive/00 Temp/hf_dep/hf_m1_sentimental_analysis', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
loaded_m1 = pipeline(task='sentiment-analysis', model='./gdrive/MyDrive/00 Temp/hf_dep/hf_m1_sentimental_analysis')

loaded_m2 = AutoModelForSequenceClassification.from_pretrained('./gdrive/MyDrive/00 Temp/hf_dep/hf_m2_text_classification', num_labels=NUM_LABELS)
loaded_t2 = BertTokenizer.from_pretrained('./gdrive/MyDrive/00 Temp/hf_dep//hf_t2_text_classification/')
# tr2 = Trainer(model=loaded_m2)

In [8]:
loaded_m1(pos_sent)

[{'label': 'POSITIVE', 'score': 0.9988293051719666}]

In [43]:
def pred_msg_cat(x, loaded_t2, tr2):
    predict_input = loaded_t2.encode_plus(x,
                                    truncation=True,
                                    max_length=512,
                                    padding=True, return_tensors="pt")
    d2 = Dataset(predict_input)
    tf_output = tr2.predict(d2)
    tf_prediction = softmax(tf_output.predictions, axis=1).numpy()[0]
    y_pred = tf_prediction.argmax()
    y_prob = tf_prediction[y_pred]
    return {"prediction" : y_pred, "probability" : y_prob}


r = pred_msg_cat(sample_x, loaded_t2, tr2)
print(r)
print(f"{r['prediction'] == sample_y}")

{'prediction': 2, 'probability': 0.9995111}
True


# BentoML

In [33]:
# reload bento_service_m2 every time to ensure updates are recorded into module 

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html # %autoreload
# %load_ext autoreload     
# %autoreload 1
# %aimport bento_service_m1
# %aimport        

# OR 

# import importlib
# import bento_service_m1
# importlib.reload(bento_service_m1)

[2021-06-21 06:02:28,485] WARNING - pip package requirement `bentoml==0.13.0` not found in current python environment


<module 'bento_service_m1' from '/content/bento_service_m1.py'>

In [10]:
warnings.filterwarnings("ignore")

In [11]:
!pip -q install bentoml "scikit-learn>=0.23.2" "urllib3==1.25.11" "folium==0.2.1"

     |████████████████████████████████| 5.0MB 4.2MB/s 
     |████████████████████████████████| 22.3MB 1.7MB/s 
     |████████████████████████████████| 133kB 58.5MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 112kB 46.8MB/s 
     |████████████████████████████████| 133kB 58.8MB/s 
     |████████████████████████████████| 133kB 54.8MB/s 
     |████████████████████████████████| 174kB 58.2MB/s 
     |████████████████████████████████| 1.3MB 39.9MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 153kB 48.5MB/s 
     |████████████████████████████████| 3.2MB 34.2MB/s 
     |████████████████████████████████| 1.3MB 41.3MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 55

## M1

In [21]:
%%writefile bento_service_m1.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput, JsonInput
from bentoml.frameworks.transformers import TransformersModelArtifact

@env(infer_pip_packages=True)
@artifacts([TransformersModelArtifact('m1')])
class SentAnalServ(BentoService):
    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        model = self.artifacts.m1.get("model")
        texts = df.to_list()
        return model(texts)

Writing bento_service_m1.py


### Test Serv - Local

In [22]:
from bento_service_m1 import SentAnalServ
sent_anal_serv = SentAnalServ()
artifact = {"model": loaded_m1, "tokenizer": loaded_m1.tokenizer}
sent_anal_serv.pack("m1", artifact)

saved_path = sent_anal_serv.save()    # save service to disk for deployment

[2021-06-21 06:33:13,369] WARNING - pip package requirement `bentoml==0.13.0` not found in current python environment
[2021-06-21 06:33:16,737] INFO - BentoService bundle 'SentAnalServ:20210621063313_DB3392' saved to: /root/bentoml/repository/SentAnalServ/20210621063313_DB3392


In [23]:
df_test = pd.DataFrame({"X" : [pos_sent, neu_sent, neg_sent], "y" : ["POSITIVE", "NEUTRAL", "NEGATIVE"]})
df_test

,X,y
0,I don't hate speaking with him,POSITIVE
1,Alvin is eating Sashimi in the living room,NEUTRAL
2,His attitude is horrible,NEGATIVE


In [24]:
sent_anal_serv.predict(df_test['X'])

[{'label': 'POSITIVE', 'score': 0.9988293051719666},
 {'label': 'NEGATIVE', 'score': 0.7627997994422913},
 {'label': 'NEGATIVE', 'score': 0.9996873736381531}]

## M2

In [12]:
%%writefile bento_service_m2.py
import torch
import pandas as pd
from tensorflow.nn import softmax
from transformers import Trainer

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import JsonInput, StringInput
from bentoml.frameworks.transformers import TransformersModelArtifact

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


@env(infer_pip_packages=True) # pip_packages=["transformers==3.1.0", "torch==1.6.0"] OR # requirements_txt_file="requirements.txt"
@artifacts([TransformersModelArtifact("m2")])
# @web_static_content('./static')       # to add custom web static content; e.g. Iris classifier: https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/static.tar.xz (https://colab.research.google.com/drive/1RNIgMBemK2O9jsLyvNqgKS6sZqCuCDif?authuser=1#scrollTo=8C86FGNZn3-s)
class MsgCatServ(BentoService):
    @api(input=JsonInput(), batch=False)
    def predict(self, msgs):
        model = self.artifacts.m2.get("model")
        tokenizer = self.artifacts.m2.get("tokenizer")

        y_preds = []
        y_probs = []
        LABELS = {0 : 'sci.crypt', 1 : 'sci.electronics', 2 : 'sci.med', 3 : 'sci.space'}

        for msg in msgs:
            tokenized_msg = tokenizer.encode_plus(msg,
                                                    truncation=True,
                                                    max_length=512,
                                                    padding=True, return_tensors="pt")
            i = Dataset(tokenized_msg)
            
            tr = Trainer(model=model)
            o = tr.predict(i)
            o_pred = softmax(o.predictions, axis=1).numpy()[0]
            y_preds.append(LABELS[o_pred.argmax()])
            y_probs.append(o_pred.max())
        output = {"result" : y_preds, "probability" : y_probs}
        return output

Writing bento_service_m2.py


### Test Serv - Local

In [38]:
from bento_service_m2 import MsgCatServ
msg_classif_serv = MsgCatServ()
artifact = {"model": loaded_m2, "tokenizer": loaded_t2}
msg_classif_serv.pack("m2", artifact)
saved_path = msg_classif_serv.save()

[2021-06-21 06:58:06,563] WARNING - pip package requirement xgboost already exist
[2021-06-21 06:58:11,192] INFO - BentoService bundle 'MsgCatServ:20210621065806_48EE89' saved to: /root/bentoml/repository/MsgCatServ/20210621065806_48EE89


In [14]:
y_pred = msg_classif_serv.predict(sample_x)
print(y_pred)

y_test = [LABELS[y] for y in sample_y]
print(y_test)

{'result': ['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space'], 'probability': [0.99963284, 0.99953604, 0.9995111, 0.99928695]}
['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']


### Test Serv - Dev Model

In [39]:
msg_classif_serv.start_dev_server(port=5000)

[2021-06-21 06:59:41,356] INFO - BentoService bundle 'MsgCatServ:20210621065806_48EE89' created at: /tmp/tmpmqj0of9z
[2021-06-21 06:59:41,383] INFO - ======= starting dev server on port: 5000 =======


In [42]:
import requests
# headers = {"content-type": "application/json"}
response = requests.post(
    "http://localhost:5000/predict",
    json=sample_x
)
print(response.text)

{"result": ["sci.crypt", "sci.electronics", "sci.med", "sci.space"], "probability": [0.9996328353881836, 0.9995360374450684, 0.9995111227035522, 0.999286949634552]}


In [43]:
msg_classif_serv.stop_dev_server()

[2021-06-21 07:00:33,497] INFO - Dev server has stopped.


### Test Serv - REST API

In [44]:
# !bentoml serve MsgCatServ:latest
!bentoml serve MsgCatServ:latest --run-with-ngrok
# sample request body : ["The best microchip in the electronic indsutry was released in June 2021."]

# now service can be called via curl, request module, browser

# curl -i --header "Content-Type: application/json" --request POST --data <data> localhost:5000/predict
# response = requests.post("http://127.0.0.1:5000/predict", json=<data> )
# browser link :  http://localhost:5000

### Test Serv - Docker

In [ ]:
# Docker isn't supported in G.Colab. Test it in local machine instead

# !bentoml containerize MsgCatServ:latest -t msg-categorizer:v1
# !docker run -p 5000:5000 msg-categorizer:v1 --workers=2

### Test Serv - Bento Service

In [45]:
import bentoml
import pandas as pd

bento_svc = bentoml.load(saved_path)

# Test loaded bentoml service:
bento_svc.predict(sample_x)

[2021-06-21 07:03:53,373] WARNING - Module `bento_service_m2` already loaded, using existing imported module.
[2021-06-21 07:03:54,799] WARNING - pip package requirement xgboost already exist


{'probability': [0.99963284, 0.99953604, 0.9995111, 0.99928695],
 'result': ['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']}

### Test Serv - PyPI
Directly installed service as Python module

In [46]:
!pip install -q {saved_path}

import MsgCatServ

installed_svc = MsgCatServ.load()
installed_svc.predict(sample_x)

{'probability': [0.99963284, 0.99953604, 0.9995111, 0.99928695],
 'result': ['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']}

### Test Serv - CLI

In [84]:
df_test = pd.DataFrame(sample_x)
df_test.to_csv("./test.csv", index=False)

In [ ]:
# !bentoml run MsgCatServ:latest predict --input '{df_test.to_json()}' --quiet
!bentoml run MsgCatServ:latest predict --input-file "./test.csv" --format "csv" --quiet

# !docker run -v $(PWD):/tmp msg-categorizer:v1  bentoml run /bento predict --input-file "<file>.csv" --format "csv" --quiet